# Flood Risk Analysis

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap

flood_df = pd.read_csv("../../data/raw/AEGISDataset.csv")
# Check the data
flood_df.head()

In [ ]:
numeric_columns = ["lat","lon","flood_heig","elevation","precipitat"]

# Convert columns to numeric, coercing errors to NaN
flood_df[numeric_columns] = flood_df[numeric_columns].apply(pd.to_numeric, errors = 'coerce')

In [ ]:
cleaned_flood_df = flood_df.dropna()

In [ ]:
# Check the cleaned data
print(f"NaN after cleaning: {cleaned_flood_df.isna().sum()}")

In [ ]:
# prepare data for heatmap
prep_flood_df = cleaned_flood_df.copy()

In [ ]:
# Normalize the flood height, elevation, and precipitation for better visualization
prep_flood_df["flood_heig_norm"] = (prep_flood_df["flood_heig"] - prep_flood_df["flood_heig"].min()) / (prep_flood_df["flood_heig"].max() - prep_flood_df["flood_heig"].min())
prep_flood_df["elevation_norm"] = (prep_flood_df["elevation"] - prep_flood_df["elevation"].min()) / (prep_flood_df["elevation"].max() - prep_flood_df["elevation"].min())
prep_flood_df["precipitat_norm"] = (prep_flood_df["precipitat"] - prep_flood_df["precipitat"].min()) / (prep_flood_df["precipitat"].max() - prep_flood_df["precipitat"].min())

In [ ]:
prep_flood_df["elevation_norm"] = 1 - prep_flood_df["elevation_norm"]

prep_flood_df["risk_score"] = (prep_flood_df["flood_heig_norm"] * 0.4 + prep_flood_df["elevation_norm"] * 0.3 +  prep_flood_df["precipitat_norm"] * 0.3)


In [ ]:
print("\nRisk Score for Each Location:")
for index, row in prep_flood_df.iterrows():
  print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}")

In [ ]:
def get_color(risk_score):
  if risk_score < 0.2:
    return 'green'
  elif 0.2 <= risk_score < 0.4:
    return 'yellow'
  elif 0.4 <= risk_score < 0.6:
    return 'orange'
  elif 0.6 <= risk_score < 0.8:
    return 'red'
  else:
    return 'darkred'

In [ ]:
prep_flood_df['risk_color'] = prep_flood_df['risk_score'].apply(get_color)


In [ ]:
manila_coordinates = [14.5995,120.9842]

In [ ]:
map_flood_risk = folium.Map(location=manila_coordinates, zoom_start=12)

heat_data = prep_flood_df[['lat','lon','risk_score']].values.tolist()

HeatMap(heat_data, radius=15, max_zoom=13).add_to(map_flood_risk)

print("\nRisk Score for Each Location:")
for index, row in prep_flood_df.iterrows():
  print(f"Latitude: {row['lat']}, Longitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}, Risk Color: {row['risk_color']}")

In [ ]:
from IPython.display import display
display(map_flood_risk)